### Подход 1: градиентный бустинг "в лоб"

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import pandas as p
from numpy import average
import sklearn
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
features = p.read_csv('./features.csv', index_col='match_id')
features_test = p.read_csv('./features_test.csv', index_col='match_id')
features_train = features.drop(columns=[
     "duration", 
     "radiant_win",
     "tower_status_radiant",
     "tower_status_dire",
     "barracks_status_dire",
     "barracks_status_radiant"
])

In [4]:
values_count = features_train.count().sort_values(ascending=True)

#### Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

In [5]:
rows = values_count.max()

print("Количество пропусков\n")
for (feature, val) in sorted(values_count.iteritems()):
    values_missing = rows - val
    if values_missing > 0:
        print(f"{feature}: {values_missing}")

Количество пропусков

dire_bottle_time: 16143
dire_courier_time: 676
dire_first_ward_time: 1826
dire_flying_courier_time: 26098
first_blood_player1: 19553
first_blood_player2: 43987
first_blood_team: 19553
first_blood_time: 19553
radiant_bottle_time: 15691
radiant_courier_time: 692
radiant_first_ward_time: 1836
radiant_flying_courier_time: 27479


Много значений пропущено в колонках, касающихся First Blood (first_blood_player1, first_blood_player2, first_blood_time, first_blood_team). Это значит, что игроки никого не убили за первые 5 минут матча.

Пропуски в *_bottle_time, *_courier_time и *_flying_courier_time означают, что соответствующая команда не купила эти предметы за первые 5 минут.

Пропуски в *_first_ward_time означают, что команда не поставила ни одного наблюдателя за 5 минут.

In [6]:
features_train = features_train.fillna(value=0)

#### 2. Как называется столбец, содержащий целевую переменную?

In [7]:
Y_col = "radiant_win"


X = features_train
y = features[Y_col]

#### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.

In [8]:
kfold = KFold(shuffle=True, n_splits=5)

Обучается классификатор с 10 деревьями
Среднее значения качества: 0.6646291455551827 (за 0:01:12.878092мс)

Обучается классификатор с 20 деревьями
Среднее значения качества: 0.6809860273308972 (за 0:02:18.501172мс)

**Обучается классификатор с 30 деревьями
Среднее значения качества: 0.6900458742897161 (за 0:03:09.979634мс)**

Обучается классификатор с 40 деревьями
Среднее значения качества: 0.694294321131233 (за 0:04:12.675311мс)

Обучается классификатор с 50 деревьями
Среднее значения качества: 0.6977442051399642 (за 0:05:26.558180мс)

Обучается классификатор с 60 деревьями
Среднее значения качества: 0.7002027254594443 (за 0:06:49.034804мс)

Обучается классификатор с 70 деревьями
Среднее значения качества: 0.7019988271689213 (за 0:07:50.472654мс)

Обучается классификатор с 80 деревьями
Среднее значения качества: 0.7047217159467425 (за 0:09:17.449093мс)

Обучается классификатор с 90 деревьями
Среднее значения качества: 0.7058723237655931 (за 0:09:42.171533мс)

Обучается классификатор с 100 деревьями
Среднее значения качества: 0.7069479623025787 (за 0:11:31.993495мс)

#### 4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

При количества деревьев больше 30 качество продолжает расти, не медленее. При 80+ рост замедляется ещё сильнее. К тому же, время на обучение начинает заметно увеличиваться.
Это значит, что оптимально испольховать 60 деревьев.

Чтобы ускорить обучение, можно использовать метод главных компонент.

### Подход 2: логистическая регрессия

Отмасштабируем признаки

In [15]:
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler()
X_scaled_array = scaler.fit_transform(X)

In [17]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from numpy import linspace
import numpy as np

def train_log_reg(c, X_train, X_test, y_train, y_test):
    ll_clf = LogisticRegressionCV(penalty='l2')
    ll_clf.C = c
    ll_clf.fit(X_train, y_train)
    y_pred = ll_clf.predict(X_test)
    score = roc_auc_score(y_test, y_pred)
    print(f"Промежуточный roc_auc_score = {score}")
    return score
        
def generate_splits():
    for i_train, i_test in kfold.split(X_scaled_array):
        X_train, X_test = X_scaled_array[i_train], X_scaled_array[i_test]
        y_train, y_test = y.iloc[i_train], y.iloc[i_test]
        yield X_train, X_test, y_train, y_test

def fit_with_cross_val(C):
    scores = [train_log_reg(C, X_train, X_test, y_train, y_test) for X_train, X_test, y_train, y_test in generate_splits()]
    return average(scores)

def find_best_c():
    best_score = None
    best_c = None
    for c in linspace(1, 10, 10):
        start_time = datetime.now()
        print(f"Обучается LogisticRegressionCV с C={c}")
        avg_score = fit_with_cross_val(c)
        if (best_score is None or avg_score > best_score):
            best_score = avg_score
            best_c = c
        exec_time = datetime.now() - start_time    
        print(f"Средний AUC-ROC = {avg_score} найден за {exec_time}мс")
    return best_score, best_c

best_score, best_c = find_best_c()
print(f"Лучшее качество логистической регрессии = {best_score} (при c={best_c})")

Обучается LogisticRegressionCV с C=1.0
Промежуточный roc_auc_score = 0.6506376765351418
Промежуточный roc_auc_score = 0.6487136119904291
Промежуточный roc_auc_score = 0.65752703357261
Промежуточный roc_auc_score = 0.6588679996655246
Промежуточный roc_auc_score = 0.6509921907025276
Средний AUC-ROC = 0.6533477024932466 найден за 0:01:26.835140мс
Обучается LogisticRegressionCV с C=2.0
Промежуточный roc_auc_score = 0.6468504612024712
Промежуточный roc_auc_score = 0.6618180359447919
Промежуточный roc_auc_score = 0.6528777425892975
Промежуточный roc_auc_score = 0.6521230674486861
Промежуточный roc_auc_score = 0.6517325802289466
Средний AUC-ROC = 0.6530803774828386 найден за 0:01:15.988866мс
Обучается LogisticRegressionCV с C=3.0
Промежуточный roc_auc_score = 0.6489177783073519
Промежуточный roc_auc_score = 0.6564415914160943
Промежуточный roc_auc_score = 0.6578466243495363
Промежуточный roc_auc_score = 0.6529422134552786
Промежуточный roc_auc_score = 0.65173621143524
Средний AUC-ROC = 0.6535

KeyboardInterrupt: 